# Assignment 1 - Part 1

In [9]:
import re
import gzip
import os
import pprint
from fnmatch import fnmatch
from bs4 import BeautifulSoup
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [2]:
INDEX_NAME = "aquaint"
DOC_TYPE = "doc"
INDEX_SETTINGS = {
    "settings" : {
        "index" : {
            "number_of_shards" : 1,
            "number_of_replicas" : 1
        }
    }
}

Add a set of documents to the index in a bulk.

In [3]:
def add_docs_bulk(es, docs):
    actions = []
    for doc_id, doc in docs.items():
        action = {
            "_index": INDEX_NAME,
            "_type": DOC_TYPE,
            "_id": doc_id,
            "_source": doc
        }
        actions.append(action)

    if len(actions) > 0:
        helpers.bulk(es, actions)

Indexes a given file

In [4]:
def index(es, file_name):
    print("Processing", file_name)
    with gzip.open(file_name, "rt") as fin:
        is_body = False
        docs = {}
        doc_id, body = None, None
        for line in fin:
            line = line.strip()
            if line.startswith("<DOCNO>"):  # get doc id
                doc_id = re.sub("<DOCNO> | </DOCNO>", "", line)
            elif line.startswith("<BODY>"):  # start to parse body
                is_body = True
                body = []
            elif line.startswith("</BODY>"):  # finished reading body
                soup = BeautifulSoup("\n".join(body), "lxml")
                headline = soup.find("headline")
                text = soup.find("text")
                docs[doc_id] = {
                    "title": headline.text if headline is not None else "",  # use an empty string if no <HEADLINE> found
                    "content": text.text if text is not None else ""  # everything inside <TEXT> is indexed as content
                }
                # get ready for next document
                doc_id = None
                is_body = False
            elif is_body:  # accumulate body content
                body.append(line)

        # bulk index the collected documents
        print("Bulk indexing", len(docs), "documents")
        add_docs_bulk(es, docs)

### Indexing

In [7]:
es = Elasticsearch()
if not es.indices.exists(INDEX_NAME):
    es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

Index a single input file

In [6]:
index(es, "data/aquaint/nyt/2000/20000101_NYT.gz")

Processing data/aquaint/nyt/2000/20000101_NYT.gz
Bulk indexing 243 documents


**TODO** Call the `index()` method on all data files in the collection.

In [8]:
root = "./data/aquaint"
suffix = "*.gz"
for path, subdirs, files in os.walk(root): # import os
    for name in files:
        if fnmatch(name, suffix):          # import fnmatch
            index(es, str(os.path.join(path, name)))

Processing ./data/aquaint\apw\1998\19980601_APW_ENG.gz
Bulk indexing 856 documents
Processing ./data/aquaint\apw\1998\19980602_APW_ENG.gz
Bulk indexing 823 documents
Processing ./data/aquaint\apw\1998\19980603_APW_ENG.gz
Bulk indexing 964 documents
Processing ./data/aquaint\apw\1998\19980604_APW_ENG.gz
Bulk indexing 1006 documents
Processing ./data/aquaint\apw\1998\19980605_APW_ENG.gz
Bulk indexing 973 documents
Processing ./data/aquaint\apw\1998\19980606_APW_ENG.gz
Bulk indexing 362 documents
Processing ./data/aquaint\apw\1998\19980607_APW_ENG.gz
Bulk indexing 407 documents
Processing ./data/aquaint\apw\1998\19980608_APW_ENG.gz
Bulk indexing 864 documents
Processing ./data/aquaint\apw\1998\19980609_APW_ENG.gz
Bulk indexing 902 documents
Processing ./data/aquaint\apw\1998\19980610_APW_ENG.gz
Bulk indexing 948 documents
Processing ./data/aquaint\apw\1998\19980611_APW_ENG.gz
Bulk indexing 951 documents
Processing ./data/aquaint\apw\1998\19980612_APW_ENG.gz
Bulk indexing 925 documents
Pro

Bulk indexing 493 documents
Processing ./data/aquaint\apw\1998\19980909_APW_ENG.gz
Bulk indexing 590 documents
Processing ./data/aquaint\apw\1998\19980910_APW_ENG.gz
Bulk indexing 563 documents
Processing ./data/aquaint\apw\1998\19980911_APW_ENG.gz
Bulk indexing 533 documents
Processing ./data/aquaint\apw\1998\19980912_APW_ENG.gz
Bulk indexing 204 documents
Processing ./data/aquaint\apw\1998\19980913_APW_ENG.gz
Bulk indexing 424 documents
Processing ./data/aquaint\apw\1998\19980914_APW_ENG.gz
Bulk indexing 511 documents
Processing ./data/aquaint\apw\1998\19980915_APW_ENG.gz
Bulk indexing 566 documents
Processing ./data/aquaint\apw\1998\19980916_APW_ENG.gz
Bulk indexing 492 documents
Processing ./data/aquaint\apw\1998\19980917_APW_ENG.gz
Bulk indexing 499 documents
Processing ./data/aquaint\apw\1998\19980918_APW_ENG.gz
Bulk indexing 568 documents
Processing ./data/aquaint\apw\1998\19980919_APW_ENG.gz
Bulk indexing 398 documents
Processing ./data/aquaint\apw\1998\19980920_APW_ENG.gz
Bulk

Bulk indexing 802 documents
Processing ./data/aquaint\apw\1998\19981217_APW_ENG.gz
Bulk indexing 759 documents
Processing ./data/aquaint\apw\1998\19981218_APW_ENG.gz
Bulk indexing 576 documents
Processing ./data/aquaint\apw\1998\19981219_APW_ENG.gz
Bulk indexing 428 documents
Processing ./data/aquaint\apw\1998\19981220_APW_ENG.gz
Bulk indexing 383 documents
Processing ./data/aquaint\apw\1998\19981221_APW_ENG.gz
Bulk indexing 480 documents
Processing ./data/aquaint\apw\1998\19981222_APW_ENG.gz
Bulk indexing 465 documents
Processing ./data/aquaint\apw\1998\19981223_APW_ENG.gz
Bulk indexing 454 documents
Processing ./data/aquaint\apw\1998\19981224_APW_ENG.gz
Bulk indexing 323 documents
Processing ./data/aquaint\apw\1998\19981225_APW_ENG.gz
Bulk indexing 261 documents
Processing ./data/aquaint\apw\1998\19981226_APW_ENG.gz
Bulk indexing 269 documents
Processing ./data/aquaint\apw\1998\19981227_APW_ENG.gz
Bulk indexing 280 documents
Processing ./data/aquaint\apw\1998\19981228_APW_ENG.gz
Bulk

Bulk indexing 353 documents
Processing ./data/aquaint\apw\1999\19990326_APW_ENG.gz
Bulk indexing 305 documents
Processing ./data/aquaint\apw\1999\19990327_APW_ENG.gz
Bulk indexing 141 documents
Processing ./data/aquaint\apw\1999\19990328_APW_ENG.gz
Bulk indexing 122 documents
Processing ./data/aquaint\apw\1999\19990329_APW_ENG.gz
Bulk indexing 259 documents
Processing ./data/aquaint\apw\1999\19990330_APW_ENG.gz
Bulk indexing 271 documents
Processing ./data/aquaint\apw\1999\19990331_APW_ENG.gz
Bulk indexing 308 documents
Processing ./data/aquaint\apw\1999\19990401_APW_ENG.gz
Bulk indexing 280 documents
Processing ./data/aquaint\apw\1999\19990402_APW_ENG.gz
Bulk indexing 250 documents
Processing ./data/aquaint\apw\1999\19990403_APW_ENG.gz
Bulk indexing 110 documents
Processing ./data/aquaint\apw\1999\19990404_APW_ENG.gz
Bulk indexing 76 documents
Processing ./data/aquaint\apw\1999\19990405_APW_ENG.gz
Bulk indexing 232 documents
Processing ./data/aquaint\apw\1999\19990406_APW_ENG.gz
Bulk 

Bulk indexing 253 documents
Processing ./data/aquaint\apw\1999\19990703_APW_ENG.gz
Bulk indexing 147 documents
Processing ./data/aquaint\apw\1999\19990704_APW_ENG.gz
Bulk indexing 150 documents
Processing ./data/aquaint\apw\1999\19990705_APW_ENG.gz
Bulk indexing 188 documents
Processing ./data/aquaint\apw\1999\19990706_APW_ENG.gz
Bulk indexing 246 documents
Processing ./data/aquaint\apw\1999\19990707_APW_ENG.gz
Bulk indexing 328 documents
Processing ./data/aquaint\apw\1999\19990708_APW_ENG.gz
Bulk indexing 314 documents
Processing ./data/aquaint\apw\1999\19990709_APW_ENG.gz
Bulk indexing 302 documents
Processing ./data/aquaint\apw\1999\19990710_APW_ENG.gz
Bulk indexing 198 documents
Processing ./data/aquaint\apw\1999\19990711_APW_ENG.gz
Bulk indexing 111 documents
Processing ./data/aquaint\apw\1999\19990712_APW_ENG.gz
Bulk indexing 287 documents
Processing ./data/aquaint\apw\1999\19990713_APW_ENG.gz
Bulk indexing 286 documents
Processing ./data/aquaint\apw\1999\19990714_APW_ENG.gz
Bulk

...

Bulk indexing 242 documents
Processing ./data/aquaint\xie\1999\19990727_XIN_ENG.gz
Bulk indexing 333 documents
Processing ./data/aquaint\xie\1999\19990728_XIN_ENG.gz
Bulk indexing 299 documents
Processing ./data/aquaint\xie\1999\19990729_XIN_ENG.gz
Bulk indexing 316 documents
Processing ./data/aquaint\xie\1999\19990730_XIN_ENG.gz
Bulk indexing 278 documents
Processing ./data/aquaint\xie\1999\19990731_XIN_ENG.gz
Bulk indexing 232 documents
Processing ./data/aquaint\xie\1999\19990801_XIN_ENG.gz
Bulk indexing 172 documents
Processing ./data/aquaint\xie\1999\19990802_XIN_ENG.gz
Bulk indexing 252 documents
Processing ./data/aquaint\xie\1999\19990803_XIN_ENG.gz
Bulk indexing 315 documents
Processing ./data/aquaint\xie\1999\19990804_XIN_ENG.gz
Bulk indexing 297 documents
Processing ./data/aquaint\xie\1999\19990805_XIN_ENG.gz
Bulk indexing 363 documents
Processing ./data/aquaint\xie\1999\19990806_XIN_ENG.gz
Bulk indexing 325 documents
Processing ./data/aquaint\xie\1999\19990807_XIN_ENG.gz
Bulk

Bulk indexing 358 documents
Processing ./data/aquaint\xie\1999\19991103_XIN_ENG.gz
Bulk indexing 336 documents
Processing ./data/aquaint\xie\1999\19991104_XIN_ENG.gz
Bulk indexing 375 documents
Processing ./data/aquaint\xie\1999\19991105_XIN_ENG.gz
Bulk indexing 330 documents
Processing ./data/aquaint\xie\1999\19991106_XIN_ENG.gz
Bulk indexing 245 documents
Processing ./data/aquaint\xie\1999\19991107_XIN_ENG.gz
Bulk indexing 198 documents
Processing ./data/aquaint\xie\1999\19991108_XIN_ENG.gz
Bulk indexing 270 documents
Processing ./data/aquaint\xie\1999\19991109_XIN_ENG.gz
Bulk indexing 328 documents
Processing ./data/aquaint\xie\1999\19991110_XIN_ENG.gz
Bulk indexing 361 documents
Processing ./data/aquaint\xie\1999\19991111_XIN_ENG.gz
Bulk indexing 351 documents
Processing ./data/aquaint\xie\1999\19991112_XIN_ENG.gz
Bulk indexing 340 documents
Processing ./data/aquaint\xie\1999\19991113_XIN_ENG.gz
Bulk indexing 237 documents
Processing ./data/aquaint\xie\1999\19991114_XIN_ENG.gz
Bulk

Bulk indexing 269 documents
Processing ./data/aquaint\xie\2000\20000210_XIN_ENG.gz
Bulk indexing 284 documents
Processing ./data/aquaint\xie\2000\20000211_XIN_ENG.gz
Bulk indexing 270 documents
Processing ./data/aquaint\xie\2000\20000212_XIN_ENG.gz
Bulk indexing 193 documents
Processing ./data/aquaint\xie\2000\20000213_XIN_ENG.gz
Bulk indexing 223 documents
Processing ./data/aquaint\xie\2000\20000214_XIN_ENG.gz
Bulk indexing 297 documents
Processing ./data/aquaint\xie\2000\20000215_XIN_ENG.gz
Bulk indexing 302 documents
Processing ./data/aquaint\xie\2000\20000216_XIN_ENG.gz
Bulk indexing 323 documents
Processing ./data/aquaint\xie\2000\20000217_XIN_ENG.gz
Bulk indexing 328 documents
Processing ./data/aquaint\xie\2000\20000218_XIN_ENG.gz
Bulk indexing 295 documents
Processing ./data/aquaint\xie\2000\20000219_XIN_ENG.gz
Bulk indexing 193 documents
Processing ./data/aquaint\xie\2000\20000220_XIN_ENG.gz
Bulk indexing 197 documents
Processing ./data/aquaint\xie\2000\20000221_XIN_ENG.gz
Bulk

Bulk indexing 363 documents
Processing ./data/aquaint\xie\2000\20000519_XIN_ENG.gz
Bulk indexing 377 documents
Processing ./data/aquaint\xie\2000\20000520_XIN_ENG.gz
Bulk indexing 212 documents
Processing ./data/aquaint\xie\2000\20000521_XIN_ENG.gz
Bulk indexing 189 documents
Processing ./data/aquaint\xie\2000\20000522_XIN_ENG.gz
Bulk indexing 356 documents
Processing ./data/aquaint\xie\2000\20000523_XIN_ENG.gz
Bulk indexing 359 documents
Processing ./data/aquaint\xie\2000\20000524_XIN_ENG.gz
Bulk indexing 373 documents
Processing ./data/aquaint\xie\2000\20000525_XIN_ENG.gz
Bulk indexing 347 documents
Processing ./data/aquaint\xie\2000\20000526_XIN_ENG.gz
Bulk indexing 325 documents
Processing ./data/aquaint\xie\2000\20000527_XIN_ENG.gz
Bulk indexing 198 documents
Processing ./data/aquaint\xie\2000\20000528_XIN_ENG.gz
Bulk indexing 190 documents
Processing ./data/aquaint\xie\2000\20000529_XIN_ENG.gz
Bulk indexing 313 documents
Processing ./data/aquaint\xie\2000\20000530_XIN_ENG.gz
Bulk

Bulk indexing 512 documents
Processing ./data/aquaint\xie\2000\20000826_XIN_ENG.gz
Bulk indexing 364 documents
Processing ./data/aquaint\xie\2000\20000827_XIN_ENG.gz
Bulk indexing 368 documents
Processing ./data/aquaint\xie\2000\20000828_XIN_ENG.gz
Bulk indexing 303 documents
Processing ./data/aquaint\xie\2000\20000829_XIN_ENG.gz
Bulk indexing 307 documents
Processing ./data/aquaint\xie\2000\20000830_XIN_ENG.gz
Bulk indexing 302 documents
Processing ./data/aquaint\xie\2000\20000831_XIN_ENG.gz
Bulk indexing 293 documents
Processing ./data/aquaint\xie\2000\20000901_XIN_ENG.gz
Bulk indexing 296 documents
Processing ./data/aquaint\xie\2000\20000902_XIN_ENG.gz
Bulk indexing 156 documents
Processing ./data/aquaint\xie\2000\20000903_XIN_ENG.gz
Bulk indexing 145 documents
Processing ./data/aquaint\xie\2000\20000904_XIN_ENG.gz
Bulk indexing 284 documents
Processing ./data/aquaint\xie\2000\20000905_XIN_ENG.gz
Bulk indexing 301 documents
Processing ./data/aquaint\xie\2000\20000906_XIN_ENG.gz
Bulk

In [11]:
test = es.get(index=INDEX_NAME, doc_type=DOC_TYPE, id="APW19980602.0001")

In [12]:
pprint.pprint(test)

{'_id': 'APW19980602.0001',
 '_index': 'aquaint',
 '_source': {'content': '\n'
                        'KIGALI, Rwanda (AP) _ A Rwandan who was convicted and '
                        'sentenced\n'
                        "to death for his role in thecountry's 1994 genocide "
                        'has attempted\n'
                        'suicide by seizing the steering wheel in a police '
                        'car, a news\n'
                        'agency reported.\n'
                        'In the Monday incident, Geoffrey Gatera, a former '
                        'surgeon,\n'
                        'seriously injured three pedestrians, but escaped '
                        'injury, the\n'
                        'private Rwandan News Agency reported.\n'
                        'Gatera was convicted Monday on charges of genocide '
                        'and crimes\n'
                        'against humanity. On the way to prison, he seizing '
                        'the